In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib as plt
import re as ree
import requests
import csv
import datetime
import time
import random

In [33]:
def vv4(row):
    row = str(row)
    X = [1,1,1]
    X[0] = row[0:4]
    X[1] = row[4:6]
    X[2] = row[6:8]
    return X[0]+'-'+X[1]+'-'+X[2]

def vv(row):
    X = row.split('-')
    return X[0]+X[1]+X[2]

def DriveYearMonthDay(d):
    d['jalaliDate'] = d['jalaliDate'].astype(str)
    d['Year'] = d['jalaliDate'].str[0:4]
    d['Month'] = d['jalaliDate'].str[4:6]
    d['Day'] = d['jalaliDate'].str[6:8]
    d['jalaliDate'] = d['jalaliDate'].astype(int)
    return d

def convert_ar_characters(input_str):

    mapping = {
        'ك': 'ک',
        'گ':'گ',
        'دِ': 'د',
        'بِ': 'ب',
        'زِ': 'ز',
        'ذِ': 'ذ',
        'شِ': 'ش',
        'سِ': 'س',
        'ى': 'ی',
        'ي': 'ی'
    }
    return _multiple_replace(mapping, input_str)


def _multiple_replace(mapping, text):
    pattern = "|".join(map(ree.escape, mapping.keys()))
    return ree.sub(pattern, lambda m: mapping[m.group()], str(text))
def vv2(row):
    X = row.split('/')
    return X[0]+X[1]+X[2]
def vv3(row):
    X = row.split('/')
    if len(X[0])<4:
        X[0] = '13' + X[0]
    if len(X[1])<2:
        X[1] = '0'+X[1]
    if len(X[2])<2:
        X[2] = '0'+X[2]
    return X[0]+X[1]+X[2]

In [34]:
path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data\\"

In [35]:
# path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data"

n = path + 'Cleaned_Stocks_Holders_1399-09-12_From94.csv'
df = pd.read_csv(n)

df = df.drop(df.loc[df['Holder']=='شخص حقیقی'].index)
df = df.drop(df[(df['Trade']=='No')&((df['close_price']==10)|(df['close_price']==1000)|(df['close_price']==10000)|(df['close_price']==100000))].index)
df = df.drop(df[(df['symbol'] == 'وقوام')&(df['close_price'] == 1000)].index)
symbols = [ 'سپرده','هما','وهنر-پذیره','نکالا','تکالا','اکالا','توسعه گردشگری ','وآفر','ودانا','نشار','نبورس','چبسپا','بدکو','چکارم','تراک','کباده','فبستم','تولیددارو','قیستو','خلیبل','پشاهن','قاروم','هوایی سامان','کورز','شلیا','دتهران','نگین','کایتا','غیوان','تفیرو','سپرمی','بتک']
df = df.drop(df[df['symbol'].isin(symbols)].index)
df = df.drop(df[df.group_name== 'صندوق سرمایه گذاری قابل معامله'].index)
df = df.drop(df[(df.symbol == 'اتکای')&(df.close_price == 1000)].index)

In [36]:
HolderData = df

In [37]:
# path = r"H:\Economics\Finance(Prof.Heidari-Aghajanzadeh)\Data"

n1 = path + 'Stocks_Prices_1399-09-12' + '.csv'
df1 = pd.read_csv(n1)

df1['jalaliDate'] = df1['jalaliDate'].apply(vv)
df = df1
symbols = [ 'سپرده','هما','وهنر-پذيره','نکالا','تکالا','اکالا','توسعه گردشگری ','وآفر','ودانا','نشار','نبورس','چبسپا','بدکو','چکارم','تراک','کباده','فبستم','تولیددارو','قیستو','خلیبل','پشاهن','قاروم','هوایی سامان','کورز','شلیا','دتهران','نگین','کایتا','غیوان','تفیرو','سپرمی','بتک']
df = df.drop(df[df['name'].isin(symbols)].index)
df = df.drop(df[df.group_name== 'صندوق سرمایه گذاری قابل معامله'].index)


df = df.drop(df[(df.name == 'اتکای')&(df.close_price == 1000)].index)
df = df.drop_duplicates()
df = df.drop(df.loc[(df['volume'] == 0)].index).sort_values(by =['name','jalaliDate']).drop(columns = ['volume','quantity']).rename(columns = {'name':'symbol'})
df = DriveYearMonthDay(df)

In [38]:
PriceData = pd.DataFrame()
PriceData = PriceData.append(df[['jalaliDate', 'date', 'symbol','close_price','group_id']])
PriceData['date1'] = PriceData['date'].apply(vv4)
PriceData['date1'] = pd.to_datetime(PriceData['date1']) 
PriceData['week_of_year'] = PriceData['date1'].dt.week 
PriceData['year_of_year'] = PriceData['date1'].dt.year
gg =PriceData.groupby('symbol')
PriceData['Ret'] = gg['close_price'].pct_change(periods = 1)*100
PriceData.head()

,jalaliDate,date,symbol,close_price,group_id,date1,week_of_year,year_of_year,Ret
1727814,13900422,20110713,آ س پ,3100.0,70,2011-07-13,28,2011,NaN
1727813,13900425,20110716,آ س پ,3252.0,70,2011-07-16,28,2011,4.903226
1727812,13900427,20110718,آ س پ,3264.0,70,2011-07-18,29,2011,0.369004
1727811,13900428,20110719,آ س پ,3203.0,70,2011-07-19,29,2011,-1.868873
1727810,13900429,20110720,آ س پ,3147.0,70,2011-07-20,29,2011,-1.748361


In [39]:
n = path + 'Factors-Daily.xlsx'
Factors = pd.read_excel(n)

In [40]:
Factors.tail()

,jalaliDate,date,SMB,HML,Winner_Loser,Market_return
2882,13990908,20201128,-0.680105,-1.598209,1.435511,1.728555
2883,13990909,20201129,-1.619483,-0.646124,-0.366239,2.616847
2884,13990910,20201130,-1.062186,1.714601,-0.160517,2.714100
2885,13990911,20201201,-0.595752,0.555956,-0.071474,0.529881
2886,13990912,20201202,0.351218,-0.764257,0.124712,-0.185540


In [41]:
PriceData = PriceData.merge(Factors,on = ['jalaliDate','date']).sort_values(by =['symbol','date']).reset_index(drop=True).dropna()

In [42]:
PriceData.head()

,jalaliDate,date,symbol,close_price,group_id,date1,week_of_year,year_of_year,Ret,SMB,HML,Winner_Loser,Market_return
1,13900425,20110716,آ س پ,3252.0,70,2011-07-16,28,2011,4.903226,0.329833,0.112264,1.671435,0.256107
2,13900427,20110718,آ س پ,3264.0,70,2011-07-18,29,2011,0.369004,-0.431230,-1.048659,4.257639,0.459158
3,13900428,20110719,آ س پ,3203.0,70,2011-07-19,29,2011,-1.868873,-1.239107,-1.245822,-1.200158,0.138590
4,13900429,20110720,آ س پ,3147.0,70,2011-07-20,29,2011,-1.748361,-3.056799,-1.671610,0.581469,0.438464
5,13900501,20110723,آ س پ,3163.0,70,2011-07-23,29,2011,0.508421,-0.453838,-0.725122,-0.621002,0.843834


In [43]:
n = path + 'RiskFree.xlsx'
df3 = pd.read_excel(n)
df3 = df3.rename(columns = {'Unnamed: 2' : 'Year'})
df3['YM'] = df3['YM'].astype(str)
df3['YM'] = df3['YM'] + '00'
df3['YM'] = df3['YM'].astype(int)
df4 = PriceData
df4['RiskFree'] = np.nan
df4['jalaliDate'] = df4['jalaliDate'].astype(int)
for i in df3.YM:
    df4.loc[df4.jalaliDate>= i,'RiskFree'] = df3.loc[df3['YM'] == i ].iloc[0,1]/12/52/7

In [44]:
PriceData = df4
PriceData['EMarketRet'] = PriceData['Market_return'] - PriceData['RiskFree']
PriceData['ERet'] = PriceData['Ret'] - PriceData['RiskFree']

In [45]:
PriceData.head()

,jalaliDate,date,symbol,close_price,group_id,date1,week_of_year,year_of_year,Ret,SMB,HML,Winner_Loser,Market_return,RiskFree,EMarketRet,ERet
1,13900425,20110716,آ س پ,3252.0,70,2011-07-16,28,2011,4.903226,0.329833,0.112264,1.671435,0.256107,0.003892,0.252215,4.899334
2,13900427,20110718,آ س پ,3264.0,70,2011-07-18,29,2011,0.369004,-0.431230,-1.048659,4.257639,0.459158,0.003892,0.455266,0.365112
3,13900428,20110719,آ س پ,3203.0,70,2011-07-19,29,2011,-1.868873,-1.239107,-1.245822,-1.200158,0.138590,0.003892,0.134698,-1.872764
4,13900429,20110720,آ س پ,3147.0,70,2011-07-20,29,2011,-1.748361,-3.056799,-1.671610,0.581469,0.438464,0.003892,0.434572,-1.752253
5,13900501,20110723,آ س پ,3163.0,70,2011-07-23,29,2011,0.508421,-0.453838,-0.725122,-0.621002,0.843834,0.003892,0.839942,0.504529


In [46]:
n1 = path + "Industry indexes 1399-09-28.csv"
df1 = pd.read_csv(n1)
df1.date = df1.date.apply(vv3)
df1 = df1.drop(columns = ['Unnamed: 0'])
df1 = df1.drop(df1[(df1.index_id == 'EWI')|(df1.index_id == 'overall_index')].index)
df1.index_id = df1.index_id.astype(int)
df1.date = df1.date.astype(int)
gg = df1.groupby('index_id')
df1['gReturn'] = gg['index'].pct_change(periods = 1)*100
fkey = zip(list(df1.index_id),list(df1.date))
mapingdict = dict(zip(fkey,df1['gReturn']))
PriceData['gEReturn'] = PriceData.set_index(['group_id','jalaliDate']).index.map(mapingdict)
PriceData['gEReturn'] = PriceData['gEReturn'] -  PriceData['RiskFree']

In [47]:
wg = PriceData.groupby(['symbol'])
def FourFactor(g):
    try:
        y = 'ERet'
        x = ['EMarketRet','HML','SMB','Winner_Loser']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['4-Residual'] = g ['ERet'] - (model.params[0] + model.params[1]*g['EMarketRet'] + model.params[2]*g['HML']  + model.params[3]*g['SMB']  + model.params[4]*g['Winner_Loser']) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g

def FiveFactor(g):
    try:
        y = 'ERet'
        x = ['EMarketRet','HML','SMB','Winner_Loser','gEReturn']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['5-Residual'] = g ['ERet'] - (model.params[0] + model.params[1]*g['EMarketRet'] + model.params[2]*g['HML']  + model.params[3]*g['SMB']  + model.params[4]*g['Winner_Loser'] + model.params[5]*g['gEReturn']) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g

def TwoFactor(g):
    try:
        y = 'ERet'
        x = ['EMarketRet','gEReturn']
        model = sm.OLS(g[y], sm.add_constant(g[x])).fit()
        g['2-Residual'] = g ['ERet'] - (model.params[0] + model.params[1]*g['EMarketRet'] + model.params[2]*g['gEReturn']) 
    except:
        print(g.symbol.iloc[0])
        g = pd.DataFrame()
    return g

re = wg.apply(FourFactor)
print('Fourfactor done')
re2 = wg.apply(FiveFactor)
re['5-Residual'] = re2['5-Residual']
print('Fivefactor done')

re2 = wg.apply(TwoFactor)
re['2-Residual'] = re2['2-Residual']
print('Twofactor done')

re = re.reset_index(drop=True)
col = 'symbol'
re[col] = re[col].apply(lambda x: convert_ar_characters(x))

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


پالايش
Fourfactor done
آتيمس
آساس
آسام
آساميد
آكورد
آوا
آگاس
ارزش
ارمغان
اطلس
اعتماد
افران
افق ملت
البرز
الماس
امين يكم
اوان
اوصتا
اپرداز
بالاس
بدكو
بذر
بميلا
بمپنا
بورس
تصميم
توسعه گردشگري 
ثروتم
حاريا
حخزر
حفاري
خاتم
خبازرس
خصدرا
دارا
دارا يكم
داريك
دمعيار
رفاه
سخند
سرو
سمگا
سپر
صايند
صنوين
فيروزا
قاسم
كاردان
كاريس
كمند
كيان
ماديرا
مبين
مدير
نگين
هاي وب
ونيرو
وهنر
ياقوت
پارند
پالايش
گدنا
گشان
گكوثر
گكيش
گنجين
گپارس
Fivefactor done
آتيمس
آساس
آسام
آساميد
آكورد
آوا
آگاس
ارزش
ارمغان
اطلس
اعتماد
افران
افق ملت
البرز
الماس
امين يكم
اوان
اوصتا
اپرداز
بالاس
بدكو
بذر
بميلا
بمپنا
بورس
تصميم
توسعه گردشگري 
ثروتم
حاريا
حخزر
حفاري
خاتم
خبازرس
خصدرا
دارا
دارا يكم
داريك
دمعيار
رفاه
سخند
سرو
سمگا
سپر
صايند
صنوين
فيروزا
قاسم
كاردان
كاريس
كمند
كيان
ماديرا
مبين
مدير
نگين
هاي وب
ونيرو
وهنر
ياقوت
پارند
پالايش
گدنا
گشان
گكوثر
گكيش
گنجين
گپارس
Twofactor done


In [48]:
residuals = re[re.jalaliDate>13940000]

In [49]:
A = set(residuals.symbol)
len(set(residuals.symbol))

699

In [50]:
C = set(HolderData.symbol)
len(set(HolderData.symbol))


612

In [51]:
# HolderData = df

In [52]:
fkey = zip(list(residuals.symbol),list(residuals.date))
mapingdict = dict(zip(fkey,residuals['4-Residual']))
HolderData['4-Residual'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)
print('4-Residual done')
fkey = zip(list(residuals.symbol),list(residuals.date))
mapingdict = dict(zip(fkey,residuals['5-Residual']))
HolderData['5-Residual'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)
print('5-Residual done')
fkey = zip(list(residuals.symbol),list(residuals.date))
mapingdict = dict(zip(fkey,residuals['2-Residual']))
HolderData['2-Residual'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)
print('2-Residual done')
fkey = zip(list(residuals.symbol),list(residuals.date))
mapingdict = dict(zip(fkey,residuals['Ret']))
HolderData['Ret'] = HolderData.set_index(['symbol','date']).index.map(mapingdict)
print('Ret done')

4-Residual done
5-Residual done
2-Residual done
Ret done


In [53]:
HolderData = HolderData.drop(HolderData[HolderData['4-Residual'].isnull()].index)

In [54]:
len(HolderData)
# [HolderData['4-Residual'].isnull()]

2627705

In [55]:

B = set(HolderData.symbol)
len(set(HolderData.symbol))

609

In [56]:
HolderData.jalaliDate.iloc[0]

13940105

In [57]:
df = HolderData

In [58]:
Holders = list(set(df.Holder))
Holders.sort()
ids = list(range(len(Holders)))
mapingdict = dict(zip(Holders, ids))
df['Holder_id'] = df['Holder'].map(mapingdict)

symbols = list(set(df.symbol))
symbols.sort()
ids = list(range(len(symbols)))
mapingdict = dict(zip(symbols, ids))
df['id'] = df['symbol'].map(mapingdict)

In [59]:
df['date1'] = df['date'].apply(vv4)
df['date1'] = pd.to_datetime(df['date1']) 
df['week_of_year'] = df['date1'].dt.week 
df['month_of_year'] = df['date1'].dt.month 
df['year_of_year'] = df['date1'].dt.year

In [60]:
df['MarketCap'] = df['close_price'] * df['shrout']

In [61]:
market = df.groupby(['date','symbol']).first().reset_index()[['jalaliDate','date','symbol','MarketCap']]
gm = market.groupby(['date'])

market['Percentile_Rank'] = gm.MarketCap.rank(pct = True) 
fkey = zip(list(market.symbol),list(market.date))
mapingdict = dict(zip(fkey,market['Percentile_Rank']))
df['Percentile_Rank'] = df.set_index(['symbol','date']).index.map(mapingdict)


In [62]:
df.head()

,date,jalaliDate,symbol,stock_id,group_name,group_id,Holder,nshares,type,Percent,...,2-Residual,Ret,Holder_id,id,date1,week_of_year,month_of_year,year_of_year,MarketCap,Percentile_Rank
2420,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صندوق بیمه اجتماعی روستاییان و عشایر,38798288.0,insurance,4.84,...,1.106068,0.88,1147,0,2015-03-25,13,3,2015,1.008800e+12,0.371542
2421,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,وغدیر,67896459.0,investment,8.48,...,1.106068,0.88,1652,0,2015-03-25,13,3,2015,1.008800e+12,0.371542
2422,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ایرانیان اطلس,121316125.0,private,15.16,...,1.106068,0.88,204,0,2015-03-25,13,3,2015,1.008800e+12,0.371542
2423,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,صنعتی و بازرگانی غدیر,164077508.0,private,20.50,...,1.106068,0.88,1264,0,2015-03-25,13,3,2015,1.008800e+12,0.371542
2424,20150325,13940105,آ س پ,1.761747e+16,انبوه سازی، املاک و مستغلات,70.0,ثاخت,321682470.0,private,40.21,...,1.106068,0.88,540,0,2015-03-25,13,3,2015,1.008800e+12,0.371542


In [28]:
# df.to_csv(path  + 'Holder_Residual.csv',index = False)
df = pd.read_csv(path + 'Holder_Residual.csv')

In [168]:
df.head()
df.loc[df.week_of_year%2 == 1,'week_of_year'] = df.loc[df.week_of_year%2 == 1]['week_of_year']-1



In [22]:
    def MonthlyCorr(f):
        
        fc = f.groupby(['year_of_year','month_of_year'])[['2-Residual_x','2-Residual_y',
                                                          '4-Residual_x','4-Residual_y',
                                                          '5-Residual_x','5-Residual_y']].corr().reset_index()
        
        TwoCor = fc.loc[fc.level_2 == '2-Residual_y'][['year_of_year','month_of_year',
                                                       '2-Residual_x']].rename(columns = {'2-Residual_x':'ρ_2'})
        FourCor = fc.loc[fc.level_2 == '4-Residual_y'][['year_of_year','month_of_year',
                                                        '4-Residual_x']].rename(columns = {'4-Residual_x':'ρ_4'})
        ThreeCor = fc.loc[fc.level_2 == '5-Residual_y'][['year_of_year','month_of_year',
                                                         '5-Residual_x']].rename(columns = {'5-Residual_x':'ρ_5'})
        
        TimeId = zip(list(TwoCor.year_of_year),list(TwoCor.month_of_year))
        mapingdict = dict(zip(TimeId,list(TwoCor.ρ_2)))
        f['Monthlyρ_2'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
        
        TimeId = zip(list(FourCor.year_of_year),list(FourCor.month_of_year))
        mapingdict = dict(zip(TimeId,list(FourCor.ρ_4)))
        f['Monthlyρ_4'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
        
        TimeId = zip(list(ThreeCor.year_of_year),list(ThreeCor.month_of_year))
        mapingdict = dict(zip(TimeId,list(ThreeCor.ρ_5)))
        f['Monthlyρ_5'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict) 
        return f
    
            
    def MonthlyCalculation(f):
    
        f = MonthlyCorr(f)        
        
        ff = f.groupby(['year_of_year','month_of_year'])[['SizeRatio','MarketCap_x',
                                                          'MarketCap_y','Percentile_Rank_x',
                                                       'Percentile_Rank_y','size1','size2',
                                                          'SameSize']].mean().reset_index()
        
        vlist = ['SizeRatio', 'MarketCap_x','MarketCap_y', 'Percentile_Rank_x', 'Percentile_Rank_y', 'size1',
               'size2', 'SameSize']
        
        for i in vlist:
            TimeId = zip(list(ff.year_of_year),list(ff.month_of_year))
            mapingdict = dict(zip(TimeId,list(ff[i])))
            f['Monthly'+i] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
        
        mfcapf =  f.groupby(['year_of_year','month_of_year'])['FCAPf','FCA'].mean().reset_index()
        TimeId = zip(list(mfcapf.year_of_year),list(mfcapf.month_of_year))
        mapingdict = dict(zip(TimeId,list(mfcapf.FCAPf)))
        f['MonthlyFCAPf'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
        TimeId = zip(list(mfcapf.year_of_year),list(mfcapf.month_of_year))
        mapingdict = dict(zip(TimeId,list(mfcapf.FCA)))
        f['MonthlyFCA'] = f.set_index(['year_of_year','month_of_year']).index.map(mapingdict)
                
        f['Monthlyρ_2_f'] = f['Monthlyρ_2'].shift(-1)
        f['Monthlyρ_4_f'] = f['Monthlyρ_4'].shift(-1)
        f['Monthlyρ_5_f'] = f['Monthlyρ_5'].shift(-1)
        return f


In [24]:
    def WeeklyCorr(f):
        fc = f.groupby(['year_of_year','week_of_year'])[['2-Residual_x','2-Residual_y',
                                                         '4-Residual_x','4-Residual_y',
                                                         '5-Residual_x','5-Residual_y']].corr().reset_index()
        
        TwoCor = fc.loc[fc.level_2 == '2-Residual_y'][['year_of_year','week_of_year',
                                                        '2-Residual_x']].rename(columns = {'2-Residual_x':'ρ_2'})
        FourCor = fc.loc[fc.level_2 == '4-Residual_y'][['year_of_year','week_of_year',
                                                        '4-Residual_x']].rename(columns = {'4-Residual_x':'ρ_4'})
        ThreeCor = fc.loc[fc.level_2 == '5-Residual_y'][['year_of_year','week_of_year',
                                                         '5-Residual_x']].rename(columns = {'5-Residual_x':'ρ_5'})
        
        TimeId = zip(list(TwoCor.year_of_year),list(TwoCor.week_of_year))
        mapingdict = dict(zip(TimeId,list(TwoCor.ρ_2)))
        f['Weeklyρ_2'] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict)
        
        TimeId = zip(list(FourCor.year_of_year),list(FourCor.week_of_year))
        mapingdict = dict(zip(TimeId,list(FourCor.ρ_4)))
        f['Weeklyρ_4'] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict)
        
        TimeId = zip(list(ThreeCor.year_of_year),list(ThreeCor.week_of_year))
        mapingdict = dict(zip(TimeId,list(ThreeCor.ρ_5)))
        f['Weeklyρ_5'] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict)
        
        return f
    
        
    def WeeklyCalculation(f):
        
        f = WeeklyCorr(f)


        ff = f.groupby(['year_of_year','week_of_year'])[['SizeRatio','MarketCap_x','MarketCap_y','Percentile_Rank_x',
                                                       'Percentile_Rank_y','size1','size2','SameSize']].mean().reset_index()
        
        vlist = ['SizeRatio', 'MarketCap_x','MarketCap_y', 'Percentile_Rank_x', 'Percentile_Rank_y', 'size1',
               'size2', 'SameSize']
        
        for i in vlist:
            TimeId = zip(list(ff.year_of_year),list(ff.week_of_year))
            mapingdict = dict(zip(TimeId,list(ff[i])))
            f['Weekly'+i] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict) 



        wfcapf =  f.groupby(['year_of_year','week_of_year'])['FCAPf','FCA'].mean().reset_index()
        TimeId = zip(list(wfcapf.year_of_year),list(wfcapf.week_of_year))
        mapingdict = dict(zip(TimeId,list(wfcapf.FCAPf)))
        f['WeeklyFCAPf'] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict)  
        TimeId = zip(list(wfcapf.year_of_year),list(wfcapf.week_of_year))
        mapingdict = dict(zip(TimeId,list(wfcapf.FCA)))
        f['WeeklyFCA'] = f.set_index(['year_of_year','week_of_year']).index.map(mapingdict) 
        f['Weeklyρ_2_f'] = f['Weeklyρ_2'].shift(-1)
        f['Weeklyρ_4_f'] = f['Weeklyρ_4'].shift(-1)
        f['Weeklyρ_5_f'] = f['Weeklyρ_5'].shift(-1)
        return f

In [25]:
def FCalculatio(a,g,S_g):
    if len(a) == 0:
        a = pd.DataFrame()
        return a
#         a = g.merge(S_g , on = ['date','jalaliDate','week_of_year','month_of_year','year_of_year'])
#         a['FCAPf'] = 0
#         a['FCA'] = 0
    else: 
        a['FCAPf'] = (a['nshares_x']*a['close_price_x'] + a['nshares_y']*a['close_price_y'])/(a['shrout_x']*a['close_price_x'] + a['shrout_y']*a['close_price_y'])
        a['FCA'] = ((a['nshares_x']*a['close_price_x'])**0.5 + (a['nshares_y']*a['close_price_y'])**0.5)/((a['shrout_x']*a['close_price_x'])**0.5 + (a['shrout_y']*a['close_price_y'])**0.5)
    return a

In [26]:
    def Calculation(g,S_g):
        intersection = list(set.intersection(set(S_g.date), set(g.date)))
        g = g.loc[g.date.isin(intersection)]
        S_g = S_g.loc[S_g.date.isin(intersection)]
        a = g.merge(S_g , on = ['Holder_id','date','jalaliDate','week_of_year','month_of_year','year_of_year'])
        a = FCalculatio(a,g,S_g)
        if len(a) == 0:
            f = pd.DataFrame()
            return f
        a['SizeRatio'] = (a['MarketCap_x'])/(a['MarketCap_y'])
        f = a.groupby(['date','jalaliDate','id_x','id_y','week_of_year','month_of_year','year_of_year',
                           'group_name_x','group_name_y','Ret_x','Ret_y','SizeRatio','MarketCap_x',
                           'MarketCap_y','2-Residual_x','2-Residual_y','4-Residual_x','4-Residual_y',
                           '5-Residual_x','5-Residual_y','Percentile_Rank_x','Percentile_Rank_y'])['FCAPf','FCA'].sum().reset_index()
        f['FCA'] = f['FCA']**2
        f['size1'] = 0
        f.loc[f.MarketCap_x>f.MarketCap_y,'size1'] = f[f.MarketCap_x>f.MarketCap_y]['Percentile_Rank_x']
        f.loc[f.MarketCap_x<f.MarketCap_y,'size1'] = f[f.MarketCap_x<f.MarketCap_y]['Percentile_Rank_y']
        f['size2'] = 0
        f.loc[f.MarketCap_x>f.MarketCap_y,'size2'] = f[f.MarketCap_x>f.MarketCap_y]['Percentile_Rank_y']
        f.loc[f.MarketCap_x<f.MarketCap_y,'size2'] = f[f.MarketCap_x<f.MarketCap_y]['Percentile_Rank_x']
        f['SameSize'] =  f['size2'] -f['size1']
        f['sgroup'] = 0
        f.loc[f.group_name_x == f.group_name_y,'sgroup'] = 1
        return f

In [31]:
gdata = df.groupby(['id'])
g = gdata.get_group(0)
S_g = gdata.get_group(74)

def FCAPf(S_g,g):
    intersection = list(set.intersection(set(S_g.date), set(g.date)))
    if len(intersection) == 0 :
#         print('0')
        return
    f= Calculation(g,S_g)
    if len (f) == 0:
#         print("No")
        return
    f = MonthlyCalculation(f)        
    f = WeeklyCalculation(f)
    return f

FCAPf(S_g,g)


,date,jalaliDate,id_x,id_y,week_of_year,month_of_year,year_of_year,group_name_x,group_name_y,Ret_x,...,WeeklyPercentile_Rank_x,WeeklyPercentile_Rank_y,Weeklysize1,Weeklysize2,WeeklySameSize,WeeklyFCAPf,WeeklyFCA,Weeklyρ_2_f,Weeklyρ_4_f,Weeklyρ_5_f
0,20150325,13940105,0,74,12,3,2015,انبوه سازی، املاک و مستغلات,انبوه سازی، املاک و مستغلات,0.880000,...,0.375913,0.446609,0.446609,0.375913,-0.070696,0.068938,0.066200,-0.772310,0.095185,-0.793644
1,20150328,13940108,0,74,12,3,2015,انبوه سازی، املاک و مستغلات,انبوه سازی، املاک و مستغلات,1.189532,...,0.375913,0.446609,0.446609,0.375913,-0.070696,0.068938,0.066200,-0.772310,0.095185,-0.793644
2,20150329,13940109,0,74,12,3,2015,انبوه سازی، املاک و مستغلات,انبوه سازی، املاک و مستغلات,4.623824,...,0.375913,0.446609,0.446609,0.375913,-0.070696,0.068938,0.066200,0.006445,0.470339,0.003778
3,20150330,13940110,0,74,14,3,2015,انبوه سازی، املاک و مستغلات,انبوه سازی، املاک و مستغلات,-1.872659,...,0.389766,0.468676,0.468676,0.389766,-0.078910,0.069190,0.066333,0.006445,0.470339,0.003778
4,20150331,13940111,0,74,14,3,2015,انبوه سازی، املاک و مستغلات,انبوه سازی، املاک و مستغلات,-0.305344,...,0.389766,0.468676,0.468676,0.389766,-0.078910,0.069190,0.066333,0.006445,0.470339,0.003778
5,20150404,13940115,0,74,14,4,2015,انبوه سازی، املاک و مستغلات,انبوه سازی، املاک و مستغلات,4.977029,...,0.389766,0.468676,0.468676,0.389766,-0.078910,0.069190,0.066333,0.006445,0.470339,0.003778
6,20150405,13940116,0,74,14,4,2015,انبوه سازی، املاک و مستغلات,انبوه سازی، املاک و مستغلات,4.449307,...,0.389766,0.468676,0.468676,0.389766,-0.078910,0.069190,0.066333,0.006445,0.470339,0.003778
7,20150406,13940117,0,74,14,4,2015,انبوه سازی، املاک و مستغلات,انبوه سازی، املاک و مستغلات,-4.259777,...,0.389766,0.468676,0.468676,0.389766,-0.078910,0.069190,0.066333,0.006445,0.470339,0.003778
8,20150407,13940118,0,74,14,4,2015,انبوه سازی، املاک و مستغلات,انبوه سازی، املاک و مستغلات,3.646973,...,0.389766,0.468676,0.468676,0.389766,-0.078910,0.069190,0.066333,0.006445,0.470339,0.003778
9,20150408,13940119,0,74,14,4,2015,انبوه سازی، املاک و مستغلات,انبوه سازی، املاک و مستغلات,-0.562984,...,0.389766,0.468676,0.468676,0.389766,-0.078910,0.069190,0.066333,0.006445,0.470339,0.003778


In [ ]:
df

In [43]:
data = pd.DataFrame()
gg = df.groupby(['id'])

for i in gg.groups.keys():
    g = gg.get_group(i)
    F_id = g.id.iloc[0]
    print('Id ' + str(F_id) , end="\r", flush=True)
    Next_df = df[df.id > F_id]
    S_gg = Next_df.groupby(['id'])
    data = data.append(S_gg.apply(FCAPf , g = g ))
    


In [55]:
d = data.reset_index(drop = True)

In [56]:
symbols = list(set(df.symbol))
symbols.sort()
ids = list(range(len(symbols)))
mapingdict = dict(zip(ids,symbols))
d['symbol_x'] = d['id_x'].map(mapingdict)
d['symbol_y'] = d['id_y'].map(mapingdict)

,date,jalaliDate,id_x,id_y,week_of_year,month_of_year,year_of_year,group_name_x,group_name_y,Ret_x,...,WeeklyFCAPf,WeeklyFCA,Monthlyρ_2_f,Monthlyρ_4_f,Monthlyρ_5_f,Weeklyρ_2_f,Weeklyρ_4_f,Weeklyρ_5_f,symbol_x,symbol_y
0,20170329,13960109,0,4,13,3,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,3.703704,...,0.042949,0.051743,0.372953,0.108111,0.390687,0.359484,0.169637,0.383524,آ س پ,آریان
1,20170403,13960114,0,4,14,4,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,0.469925,...,0.042896,0.051636,0.372953,0.108111,0.390687,0.359484,0.169637,0.383524,آ س پ,آریان
2,20170404,13960115,0,4,14,4,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-1.309635,...,0.042896,0.051636,0.372953,0.108111,0.390687,0.359484,0.169637,0.383524,آ س پ,آریان
3,20170405,13960116,0,4,14,4,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,-2.654028,...,0.042896,0.051636,0.372953,0.108111,0.390687,0.359484,0.169637,0.383524,آ س پ,آریان
4,20170408,13960119,0,4,14,4,2017,انبوه سازی، املاک و مستغلات,سرمایه گذاریها,0.876339,...,0.042896,0.051636,0.372953,0.108111,0.390687,0.359484,0.169637,0.383524,آ س پ,آریان


In [57]:
def add(row):
    if len(row)<2:
        row = '0'+ row
    return row
d['month_of_year'] = d['month_of_year'].astype(str).apply(add )
d['week_of_year'] = d['week_of_year'].astype(str).apply(add )


d['year_of_year'] = d['year_of_year'].astype(str)

d['Year_Month_week'] = d['year_of_year'] + d['month_of_year'] + d['week_of_year']
d['Year_Month'] = d['year_of_year'] + d['month_of_year']

days = list(set(d.date))
days.sort()
t = list(range(len(days)))
mapingdict = dict(zip(days, t))
d['t'] = d['date'].map(mapingdict)

days = list(set(d.Year_Month_week))
days.sort()
t = list(range(len(days)))
mapingdict = dict(zip(days, t))
d['t_Week'] = d['Year_Month_week'].map(mapingdict)

days = list(set(d.Year_Month))
days.sort()
t = list(range(len(days)))
mapingdict = dict(zip(days, t))
d['t_Month'] = d['Year_Month'].map(mapingdict)


d['id'] = d['symbol_x'] + d['symbol_y']
ids = list(set(d.id))
ids.sort()
id = list(range(len(ids)))
mapingdict = dict(zip(ids, id))
d['id'] = d['id'].map(mapingdict)

def NormalTransform(df_sub):
    col = df_sub.transform('rank')
    return (col-col.mean())/col.std()


gg = d.groupby(['t'])
d['FCAP*'] = gg['FCAPf'].apply(NormalTransform)
d['FCA*'] = gg['FCA'].apply(NormalTransform)

gg = d.groupby(['t_Week'])
d['WeeklyFCAP*'] = gg['WeeklyFCAPf'].apply(NormalTransform)
d['WeeklyFCA*'] = gg['WeeklyFCA'].apply(NormalTransform)

gg = d.groupby(['t_Month'])
d['MonthlyFCAP*'] = gg['MonthlyFCAPf'].apply(NormalTransform) 
d['MonthlyFCA*'] = gg['MonthlyFCA'].apply(NormalTransform)

In [60]:
d.head().columns
n3 = path + 'NormalzedFCAP4.0.csv'
d.to_csv(n3,index = False)
print('d done')
a = d.drop_duplicates(['t','id'])
n3 = path + 'NormalzedFCAP4.1.csv'
a.to_csv(n3,index = False)

Montly =  a.drop_duplicates(['id','t_Month'],keep = "last")[['date', 'jalaliDate', 'id_x', 'id_y', 'group_name_x',
                                                            'group_name_y',
       'SizeRatio', 'MarketCap_x', 'MarketCap_y', 'Percentile_Rank_x',
       'Percentile_Rank_y', 'size1', 'size2', 'SameSize', 'sgroup',
       'Monthlyρ_2', 'Monthlyρ_4', 'Monthlyρ_5', 'MonthlySizeRatio',
       'MonthlyMarketCap_x', 'MonthlyMarketCap_y', 'MonthlyPercentile_Rank_x',
       'MonthlyPercentile_Rank_y', 'Monthlysize1', 'Monthlysize2',
       'MonthlySameSize', 'MonthlyFCAPf', 'MonthlyFCA', 'Monthlyρ_2_f',
       'Monthlyρ_4_f', 'Monthlyρ_5_f', 'symbol_x', 'symbol_y', 'Year_Month',
       't_Month', 'id', 'MonthlyFCAP*', 'MonthlyFCA*']] 
n3 = path + 'MonthlyNormalzedFCAP4.1.csv'
Montly.to_csv(n3,index = False)

Weekly = a.drop_duplicates(['id','t_Week'],keep = "last")[['date', 'jalaliDate', 'id_x', 'id_y', 'group_name_x',
                                                           'group_name_y',
       'SizeRatio', 'MarketCap_x', 'MarketCap_y', 'Percentile_Rank_x',
       'Percentile_Rank_y', 'size1', 'size2', 'SameSize', 'sgroup',
       'Weeklyρ_2', 'Weeklyρ_4', 'Weeklyρ_5', 'WeeklySizeRatio',
       'WeeklyMarketCap_x', 'WeeklyMarketCap_y', 'WeeklyPercentile_Rank_x',
       'WeeklyPercentile_Rank_y', 'Weeklysize1', 'Weeklysize2',
       'WeeklySameSize', 'WeeklyFCAPf', 'WeeklyFCA', 'Weeklyρ_2_f',
       'Weeklyρ_4_f', 'Weeklyρ_5_f', 'symbol_x', 'symbol_y', 'Year_Month_week',
       't_Week', 'id', 'WeeklyFCAP*', 'WeeklyFCA*']]
n3 = path + 'WeeklyNormalzedFCAP4.1.csv'
Weekly.to_csv(n3,index = False)


d done


In [ ]:

Rdata = pd.DataFrame()
Rdata = gg.apply(FCAPf ,GData = GData, Pairs = Pairs,dgg = dgg)

In [1]:
a

NameError: name 'a' is not defined